# HTTP Requests
---
This notebook introduces the `requests` library, which we can use to retrieve data from the web.

Some data resources are provided online as **APIs** (Application Programming Interfaces), which can be accessed using a protocol called HTTP (HyperText Transfer Protocol). These interfaces may allow us to retrieve information in various ways, for example by asking for a particular record or making a search.

In this notebook we will look at an API provided by the [US Geological Survey](https://www.usgs.gov/programs/earthquake-hazards) providing continually-updated information about earthquakes around the world.

Firstly, take a look at the web form for searching manually. Go to https://earthquake.usgs.gov/earthquakes/search/

Notice all of the different search options and try some searches out.

The results are presented as a list and an interactive map.

<img src='../resources/earthquakes.png'>

---
## Making a request
But what if we want to retrieve the data themselves? Here is an example using `requests`:

In [1]:
import requests

# An endpoint is a URL that accepts requests
endpoint = "https://earthquake.usgs.gov/fdsnws/event/1/query?"

# The parameters for our search, in the form of a dictionary
ps = { "format": "csv", 
       "starttime": "2023-09-27", 
       "endtime": "2023-09-28",
       "minmagnitude": 4.5  
     }

response = requests.get(endpoint, params=ps)

print(response.text)


time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
2023-09-27T17:47:21.700Z,0.874,93.1233,10,5.4,mww,81,58,4.472,0.72,us,us6000lb60,2023-09-27T18:11:02.504Z,"off the west coast of northern Sumatra",earthquake,6.57,1.678,0.08,15,reviewed,us,us
2023-09-27T16:15:22.217Z,-3.3735,128.8414,33.157,5,mb,63,44,1.558,0.61,us,us6000lb4e,2023-09-27T16:42:05.040Z,"9 km WSW of Amahai, Indonesia",earthquake,6.65,4.717,0.059,93,reviewed,us,us
2023-09-27T09:51:05.009Z,56.4485,-168.2526,5.637,4.8,mb,133,45,0.734,0.51,us,us6000lb2a,2023-09-27T16:56:38.998Z,"81 km ESE of Saint George, Alaska",earthquake,5.64,4.66,0.057,95,reviewed,us,us
2023-09-27T08:38:43.023Z,-18.8247,169.2845,111.712,4.6,mb,21,159,4.143,0.45,us,us6000lb27,2023-09-27T11:15:28.040Z,"79 km N of Isangel, Vanuatu",earthquake,6.05,8.459,0.132,17,reviewed,us,us
2023-09-27T05:40:03.161Z,-4.4378,153.0896,10,4.5,mb,38,124,0.955,0.5,us,us6

An *endpoint* is a particular URL (web address) that will accept a request in a particular format and return a *response*. 

The USGS endpoint is able to return data in CSV format, which we can access using the `.text` attribute.

The parameter options for this endpoint are explained 
[here](https://earthquake.usgs.gov/fdsnws/event/1/?ref=springboard).

---
## Converting to a DataFrame
If we want to create a DataFrame from this CSV text, we can do the following:

In [2]:
import pandas as pd
import io

# make a string buffer from the CSV text
buf = io.StringIO(response.text) 

# pandas can now read the data from the buffer
data = pd.read_csv(buf)          

data

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2023-09-27T17:47:21.700Z,0.8740,93.1233,10.000,5.4,mww,81,58,4.472,0.72,...,2023-09-27T18:11:02.504Z,off the west coast of northern Sumatra,earthquake,6.57,1.678,0.080,15,reviewed,us,us
1,2023-09-27T16:15:22.217Z,-3.3735,128.8414,33.157,5.0,mb,63,44,1.558,0.61,...,2023-09-27T16:42:05.040Z,"9 km WSW of Amahai, Indonesia",earthquake,6.65,4.717,0.059,93,reviewed,us,us
2,2023-09-27T09:51:05.009Z,56.4485,-168.2526,5.637,4.8,mb,133,45,0.734,0.51,...,2023-09-27T16:56:38.998Z,"81 km ESE of Saint George, Alaska",earthquake,5.64,4.660,0.057,95,reviewed,us,us
3,2023-09-27T08:38:43.023Z,-18.8247,169.2845,111.712,4.6,mb,21,159,4.143,0.45,...,2023-09-27T11:15:28.040Z,"79 km N of Isangel, Vanuatu",earthquake,6.05,8.459,0.132,17,reviewed,us,us
4,2023-09-27T05:40:03.161Z,-4.4378,153.0896,10.000,4.5,mb,38,124,0.955,0.50,...,2023-09-27T06:37:43.040Z,"91 km E of Kokopo, Papua New Guinea",earthquake,7.66,1.889,0.094,34,reviewed,us,us
5,2023-09-27T02:54:45.292Z,31.4758,141.8182,17.200,5.0,mb,60,120,2.357,0.64,...,2023-09-27T03:15:25.040Z,"Izu Islands, Japan region",earthquake,9.99,4.905,0.040,205,reviewed,us,us
6,2023-09-27T02:14:27.605Z,54.9980,-160.4559,49.669,5.1,mb,95,154,0.352,0.93,...,2023-09-27T15:30:18.040Z,"37 km S of Sand Point, Alaska",earthquake,4.85,5.561,0.050,128,reviewed,us,us


---
### JSON

We are getting comfortable with CSV format, but there are many other data formats that are in common use.

When working with APIs, we often encounter the **JSON** (JavaScript Object Notation) format. 

The USGS endpoint can return responses in [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON), a particular flavour of JSON which is used to describe geolocated objects.

In [3]:
endpoint = "https://earthquake.usgs.gov/fdsnws/event/1/query?"

ps = { "format": "geojson", 
       "starttime": "2023-09-27", 
       "endtime": "2023-09-28",
       "minmagnitude": 4.8  
     }

response = requests.get(endpoint, params=ps)

print(response.text)


{"type":"FeatureCollection","metadata":{"generated":1695838381000,"url":"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-09-27&endtime=2023-09-28&minmagnitude=4.8","title":"USGS Earthquakes","status":200,"api":"1.14.0","count":5},"features":[{"type":"Feature","properties":{"mag":5.4,"place":"off the west coast of northern Sumatra","time":1695836841700,"updated":1695838262504,"tz":null,"url":"https://earthquake.usgs.gov/earthquakes/eventpage/us6000lb60","detail":"https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us6000lb60&format=geojson","felt":null,"cdi":null,"mmi":0,"alert":"green","status":"reviewed","tsunami":0,"sig":449,"net":"us","code":"6000lb60","ids":",usauto6000lb60,pt23270000,us6000lb60,","sources":",usauto,pt,us,","types":",internal-moment-tensor,internal-origin,losspager,origin,phase-data,shakemap,","nst":81,"dmin":4.472,"rms":0.72,"gap":58,"magType":"mww","type":"earthquake","title":"M 5.4 - off the west coast of northern Sumatra"},"g

Notice that JSON data consists of nested dictionaries (marked by **{ }**) and lists (marked by **\[ ]**).
It is a very flexible format, and can be used to describe data structures that are more complex than just tables.


If you need to work with JSON data, the built-in `json` module is very handy.

In [4]:
import json
data_dict = json.loads(response.text)
data_dict

{'type': 'FeatureCollection',
 'metadata': {'generated': 1695838381000,
  'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-09-27&endtime=2023-09-28&minmagnitude=4.8',
  'title': 'USGS Earthquakes',
  'status': 200,
  'api': '1.14.0',
  'count': 5},
 'features': [{'type': 'Feature',
   'properties': {'mag': 5.4,
    'place': 'off the west coast of northern Sumatra',
    'time': 1695836841700,
    'updated': 1695838262504,
    'tz': None,
    'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us6000lb60',
    'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us6000lb60&format=geojson',
    'felt': None,
    'cdi': None,
    'mmi': 0,
    'alert': 'green',
    'status': 'reviewed',
    'tsunami': 0,
    'sig': 449,
    'net': 'us',
    'code': '6000lb60',
    'ids': ',usauto6000lb60,pt23270000,us6000lb60,',
    'sources': ',usauto,pt,us,',
    'types': ',internal-moment-tensor,internal-origin,losspager,origin,phase-data,shakem

*data_dict* is now just a python dictionary, which we can manipulate as we like.
For example, to print the magnitude of all earthquakes in the response:

In [5]:
for eq in data_dict['features']:
    print(eq['properties']['mag'])

5.4
5
4.8
5
5.1


If needed, we can also convert a JSON dictionary to a pandas DataFrame using [`json_normalize`](https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html#pandas.json_normalize):

In [6]:
# the 'record_path' argument shows how to locate the table rows within the dict
pd.json_normalize(data_dict, record_path=['features'])

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,Feature,us6000lb60,5.4,off the west coast of northern Sumatra,1695836841700,1695838262504,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",internal-moment-tensor,internal-origin,losspa...",81,4.472,0.72,58,mww,earthquake,M 5.4 - off the west coast of northern Sumatra,Point,"[93.1233, 0.874, 10]"
1,Feature,us6000lb4e,5.0,"9 km WSW of Amahai, Indonesia",1695831322217,1695832925040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",63,1.558,0.61,44,mb,earthquake,"M 5.0 - 9 km WSW of Amahai, Indonesia",Point,"[128.8414, -3.3735, 33.157]"
2,Feature,us6000lb2a,4.8,"81 km ESE of Saint George, Alaska",1695808265009,1695833798998,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",moment-tensor,origin,phase-data,shakemap,",133,0.734,0.51,45,mb,earthquake,"M 4.8 - 81 km ESE of Saint George, Alaska",Point,"[-168.2526, 56.4485, 5.637]"
3,Feature,us6000lb14,5.0,"Izu Islands, Japan region",1695783285292,1695784525040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",60,2.357,0.64,120,mb,earthquake,"M 5.0 - Izu Islands, Japan region",Point,"[141.8182, 31.4758, 17.2]"
4,Feature,us6000lb0w,5.1,"37 km S of Sand Point, Alaska",1695780867605,1695828618040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,4.0,...,",dyfi,moment-tensor,origin,phase-data,shakemap,",95,0.352,0.93,154,mb,earthquake,"M 5.1 - 37 km S of Sand Point, Alaska",Point,"[-160.4559, 54.998, 49.669]"
